# Movies recommandation

## 1. Imports

### 1.1 Libraries

In [1]:
# builtin
import os, time, sys, random

# data
import pandas as pd
import numpy as np
import requests
import math

# viz
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# NLP
import string
import nltk
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.corpus import words
from nltk.corpus import RegexpTokenizer
from nltk.stem.snowball import FrenchStemmer
from collections import Counter

# ML
from gensim.models import Word2Vec
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

# other
import warnings
warnings.filterwarnings("ignore")
from pandarallel import pandarallel

### 1.2 Download and options

In [2]:
nltk.download('stopwords')
nltk.download('wordpunct')
nltk.download('wordnet')
nltk.download('words')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\melvin.derouk\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Error loading wordpunct: Package 'wordpunct' not found in
[nltk_data]     index
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\melvin.derouk\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\melvin.derouk\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\melvin.derouk\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
sns.set()

In [4]:
pandarallel.initialize()

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


### 1.3 Loading data

In [6]:
# CSV

#Env Perso
#df = pd.read_csv(r"C:\Users\derou\OneDrive\Bureau\DATA\PORTFOLIO\Recommandation de films\df_movies_cleaned.csv")

# Env Vinci
df = pd.read_csv(r"C:\Users\melvin.derouk\Desktop\Data formation\Movies-Recommandations\df_movies_cleaned.csv")

## 2. Work on a specific document

In [7]:
# Fonction de saut de ligne pour output
def insert_newlines(string, every=80):
    lines = []
    for i in range(0, len(string), every):
        lines.append(string[i:i+every])
    return '\n'.join(lines)

In [8]:
doc = df.Synopsis.sample(1)
doc = doc.values[0]
print(insert_newlines(doc, every=200))

Rowena, une fillette de 11 ans pleine d'entrain, est confrontée au divorce difficile de ses parents. Espérant que les choses redeviennent comme elles étaient avant que la nouvelle petite amie de son p
ère et son fils n'entrent en scène, Rowena fait un vœu au Père Noël d'un centre commercial. Cependant, ses souhaits de Noël tournent mal quand elle se retrouve à vivre le même jour encore et encore. C
oincée dans cette boucle sans fin, elle doit apprendre à aimer sa nouvelle famille recomposée et apprendre le vrai sens de Noël.


### 2.1 Lower

In [9]:
doc = doc.lower()

In [10]:
print(insert_newlines(doc, every=200))

rowena, une fillette de 11 ans pleine d'entrain, est confrontée au divorce difficile de ses parents. espérant que les choses redeviennent comme elles étaient avant que la nouvelle petite amie de son p
ère et son fils n'entrent en scène, rowena fait un vœu au père noël d'un centre commercial. cependant, ses souhaits de noël tournent mal quand elle se retrouve à vivre le même jour encore et encore. c
oincée dans cette boucle sans fin, elle doit apprendre à aimer sa nouvelle famille recomposée et apprendre le vrai sens de noël.


### 2.2 Tokenization

In [11]:
tokens = word_tokenize(doc)
tokens

['rowena',
 ',',
 'une',
 'fillette',
 'de',
 '11',
 'ans',
 'pleine',
 "d'entrain",
 ',',
 'est',
 'confrontée',
 'au',
 'divorce',
 'difficile',
 'de',
 'ses',
 'parents',
 '.',
 'espérant',
 'que',
 'les',
 'choses',
 'redeviennent',
 'comme',
 'elles',
 'étaient',
 'avant',
 'que',
 'la',
 'nouvelle',
 'petite',
 'amie',
 'de',
 'son',
 'père',
 'et',
 'son',
 'fils',
 "n'entrent",
 'en',
 'scène',
 ',',
 'rowena',
 'fait',
 'un',
 'vœu',
 'au',
 'père',
 'noël',
 "d'un",
 'centre',
 'commercial',
 '.',
 'cependant',
 ',',
 'ses',
 'souhaits',
 'de',
 'noël',
 'tournent',
 'mal',
 'quand',
 'elle',
 'se',
 'retrouve',
 'à',
 'vivre',
 'le',
 'même',
 'jour',
 'encore',
 'et',
 'encore',
 '.',
 'coincée',
 'dans',
 'cette',
 'boucle',
 'sans',
 'fin',
 ',',
 'elle',
 'doit',
 'apprendre',
 'à',
 'aimer',
 'sa',
 'nouvelle',
 'famille',
 'recomposée',
 'et',
 'apprendre',
 'le',
 'vrai',
 'sens',
 'de',
 'noël',
 '.']

In [12]:
# longueur de la liste
len(tokens)

99

In [13]:
# longueur de la liste (sans les doublons)
len(set(tokens))

72

In [14]:
def display_tokens_infos(tokens):
    """display info about corpus"""

    print(f"nb tokens {len(tokens)}, nb tokens uniques {len(set(tokens))}")
    print(tokens[:30])

In [15]:
tokens = wordpunct_tokenize(doc)
display_tokens_infos(tokens)

nb tokens 105, nb tokens uniques 74
['rowena', ',', 'une', 'fillette', 'de', '11', 'ans', 'pleine', 'd', "'", 'entrain', ',', 'est', 'confrontée', 'au', 'divorce', 'difficile', 'de', 'ses', 'parents', '.', 'espérant', 'que', 'les', 'choses', 'redeviennent', 'comme', 'elles', 'étaient', 'avant']


### 2.3 Stopwords

In [16]:
stop_words = set(stopwords.words('french'))

In [17]:
tokens = [w for w in tokens if w not in stop_words]
display_tokens_infos(tokens)

nb tokens 68, nb tokens uniques 52
['rowena', ',', 'fillette', '11', 'ans', 'pleine', "'", 'entrain', ',', 'confrontée', 'divorce', 'difficile', 'parents', '.', 'espérant', 'choses', 'redeviennent', 'comme', 'elles', 'avant', 'nouvelle', 'petite', 'amie', 'père', 'fils', "'", 'entrent', 'scène', ',', 'rowena']


In [18]:
tokenizer = nltk.RegexpTokenizer(r'\w+')
tokens = tokenizer.tokenize(doc)
display_tokens_infos(tokens)

nb tokens 93, nb tokens uniques 71
['rowena', 'une', 'fillette', 'de', '11', 'ans', 'pleine', 'd', 'entrain', 'est', 'confrontée', 'au', 'divorce', 'difficile', 'de', 'ses', 'parents', 'espérant', 'que', 'les', 'choses', 'redeviennent', 'comme', 'elles', 'étaient', 'avant', 'que', 'la', 'nouvelle', 'petite']


In [19]:
tokens = [w for w in tokens if w not in stop_words]
display_tokens_infos(tokens)

nb tokens 56, nb tokens uniques 49
['rowena', 'fillette', '11', 'ans', 'pleine', 'entrain', 'confrontée', 'divorce', 'difficile', 'parents', 'espérant', 'choses', 'redeviennent', 'comme', 'elles', 'avant', 'nouvelle', 'petite', 'amie', 'père', 'fils', 'entrent', 'scène', 'rowena', 'fait', 'vœu', 'père', 'noël', 'centre', 'commercial']


### 2.4 First cleaning function

In [20]:
def process_synopsis_1(doc, rejoin=False):

    # lower
    doc = doc.lower().strip()

    # tokenize
    tokenizer = RegexpTokenizer(r'\w+')
    raw_tokens_list = tokenizer.tokenize(doc)

    # stop words
    cleaned_tokens_list = [w for w in raw_tokens_list if w not in stop_words]

    if rejoin : 
        return " ".join(cleaned_tokens_list)
    
    return cleaned_tokens_list

In [21]:
tokens = process_synopsis_1(doc)
display_tokens_infos(tokens)

nb tokens 56, nb tokens uniques 49
['rowena', 'fillette', '11', 'ans', 'pleine', 'entrain', 'confrontée', 'divorce', 'difficile', 'parents', 'espérant', 'choses', 'redeviennent', 'comme', 'elles', 'avant', 'nouvelle', 'petite', 'amie', 'père', 'fils', 'entrent', 'scène', 'rowena', 'fait', 'vœu', 'père', 'noël', 'centre', 'commercial']


## 3. Work on the entire corpus

### 3.1 Build raw corpus

In [22]:
raw_corpus = "".join(df.Synopsis.values)
raw_corpus[:3_00]

'Un groupe d\'animaux animatroniques interprète des chansons pour enfants le jour et fait des razzias meurtrières la nuit. Adaptation du jeu vidéo "Five Nights at Freddy\'s", au croisement du Survival Horror - action - stratégie.Dans l\'espoir d\'une guérison miraculeuse, John Kramer se rend au Mexique p'

In [23]:
len(raw_corpus)

3707250

In [24]:
corpus = process_synopsis_1(raw_corpus)
display_tokens_infos(corpus)

nb tokens 367137, nb tokens uniques 37200
['groupe', 'animaux', 'animatroniques', 'interprète', 'chansons', 'enfants', 'jour', 'fait', 'razzias', 'meurtrières', 'nuit', 'adaptation', 'jeu', 'vidéo', 'five', 'nights', 'at', 'freddy', 'croisement', 'survival', 'horror', 'action', 'stratégie', 'espoir', 'guérison', 'miraculeuse', 'john', 'kramer', 'rend', 'mexique']


In [25]:
tmp = pd.Series(corpus).value_counts()
tmp

a             3681
plus          3130
jeune         2010
vie           1888
alors         1751
              ... 
medecine         1
petillante       1
complicite       1
immediate        1
bohême           1
Length: 37200, dtype: int64

In [26]:
tmp.head(20)

a          3681
plus       3130
jeune      2010
vie        1888
alors      1751
deux       1649
tout       1617
va         1528
après      1351
faire      1291
monde      1248
ans        1221
femme      1191
fait       1186
cette      1180
où         1172
leurs      1161
être       1086
famille    1019
homme       971
dtype: int64

In [27]:
tmp.tail(10)

indissolublement    1
inéxorablement      1
mure                1
jawbreaker          1
jullie              1
medecine            1
petillante          1
complicite          1
immediate           1
bohême              1
dtype: int64

In [28]:
tmp.describe()

count    37200.000000
mean         9.869274
std         52.656774
min          1.000000
25%          1.000000
50%          2.000000
75%          5.000000
max       3681.000000
dtype: float64

### 3.2 List rare tokens

In [29]:
# unique words = usefull ?

tmp = pd.Series(corpus).value_counts()
list_unique_words = tmp[tmp==1]
list_unique_words[:20]

beery           1
tilson          1
hynde           1
sautant         1
brentley        1
mobil           1
anticipée       1
ringo           1
pampa           1
vacancier       1
existentiels    1
railroad        1
tigers          1
shandong        1
morse           1
yasar           1
whelan          1
fixation        1
stratford       1
witherspoon     1
dtype: int64

In [30]:
len(list_unique_words)

16176

In [31]:
list_unique_words = list(list_unique_words.index)
list_unique_words[:20]

['beery',
 'tilson',
 'hynde',
 'sautant',
 'brentley',
 'mobil',
 'anticipée',
 'ringo',
 'pampa',
 'vacancier',
 'existentiels',
 'railroad',
 'tigers',
 'shandong',
 'morse',
 'yasar',
 'whelan',
 'fixation',
 'stratford',
 'witherspoon']

In [32]:
tmp = pd.DataFrame({"words" : list_unique_words})
tmp.to_csv("unique_words.csv", index=False)

In [33]:
# idem for min 5 times

tmp = pd.Series(corpus).value_counts()
list_min_5_words = tmp[tmp==5]
list_min_5_words[:20]

débarquement     5
apte             5
capulet          5
redonne          5
wyoming          5
manquer          5
raoul            5
soigneusement    5
ramenés          5
jardins          5
peterson         5
maternité        5
ku               5
désirent         5
piller           5
festive          5
adrian           5
inquiétude       5
chambouler       5
ferroviaire      5
dtype: int64

In [34]:
len(list_min_5_words)

1409

In [35]:
list_min_5_words = list(list_min_5_words.index)
list_min_5_words[:20]

['débarquement',
 'apte',
 'capulet',
 'redonne',
 'wyoming',
 'manquer',
 'raoul',
 'soigneusement',
 'ramenés',
 'jardins',
 'peterson',
 'maternité',
 'ku',
 'désirent',
 'piller',
 'festive',
 'adrian',
 'inquiétude',
 'chambouler',
 'ferroviaire']

In [36]:
tmp = pd.DataFrame({"words" : list_min_5_words})
tmp.to_csv("min_5_words.csv", index=False)

In [37]:
# idem for min 10 times ???????????????????

tmp = pd.Series(corpus).value_counts()
list_min_10_words = tmp[tmp==10]
list_min_10_words[:20]

déchirée        10
cambrioleurs    10
financières     10
arriveront      10
racines         10
jeter           10
partagé         10
voue            10
balboa          10
prostitution    10
wells           10
changeant       10
accumulent      10
traîneau        10
mae             10
ennuyeuse       10
défier          10
projeté         10
futures         10
inexpliqués     10
dtype: int64

In [38]:
len(list_min_10_words)

416

### 3.3 Second cleaning function

In [39]:
def process_synopsis_2(doc,
                       rejoin=False,
                       list_rare_words=None,
                       min_len_word=3,
                       force_is_alpha=True) : 
    
    """cf process_synopsis_1 but with list_unique_words, min_len_word, and force_is_alpha

    positionnal arguments :
    ------------------------
    doc : str : the document (aka a text in str format) to process

    opt args : 
    ------------------------
    rejoin : bool : if True return a string else return the list of tokens
    list_rare_words : list : a list of rare words to exclude
    min_len_word : int : minimum lenght of a word to not exclude
    force_is_alpha : if 1, exclude all tokens with a numeric character

    return : 
    ------------------------
    a string (if rejoin is True) or a list of tokens
    """
    
    # list unique words
    if not list_rare_words:
        list_rare_words = []

    # lower
    doc = doc.lower().strip()

    # tokenize
    tokenizer = RegexpTokenizer(r'\w+')
    raw_tokens_list = tokenizer.tokenize(doc)

    # stop words
    cleaned_tokens_list = [w for w in raw_tokens_list if w not in stop_words]

    #############################################################
    #############################################################

    # no rare tokens
    non_rare_tokens = [w for w in cleaned_tokens_list if w not in list_rare_words]

    # no more len words
    more_than_N = [w for w in non_rare_tokens if len(w) >= min_len_word]

    # only alpha chars
    if force_is_alpha : 
        alpha_tokens = [w for w in more_than_N if w.isalpha()]
    else : 
        alpha_tokens = more_than_N

    #############################################################
    #############################################################

    # manage return type
    if rejoin : 
        return " ".join(alpha_tokens)
    
    return alpha_tokens

In [40]:
display_tokens_infos(corpus)

nb tokens 367137, nb tokens uniques 37200
['groupe', 'animaux', 'animatroniques', 'interprète', 'chansons', 'enfants', 'jour', 'fait', 'razzias', 'meurtrières', 'nuit', 'adaptation', 'jeu', 'vidéo', 'five', 'nights', 'at', 'freddy', 'croisement', 'survival', 'horror', 'action', 'stratégie', 'espoir', 'guérison', 'miraculeuse', 'john', 'kramer', 'rend', 'mexique']


In [41]:
len(set(corpus))

37200

In [42]:
#3-5 min process

corpus = process_synopsis_2(raw_corpus,
                            list_rare_words=list_unique_words,
                            rejoin=False)
display_tokens_infos(corpus)

nb tokens 338579, nb tokens uniques 20602
['groupe', 'animaux', 'animatroniques', 'interprète', 'chansons', 'enfants', 'jour', 'fait', 'meurtrières', 'nuit', 'adaptation', 'jeu', 'vidéo', 'five', 'freddy', 'croisement', 'action', 'stratégie', 'espoir', 'guérison', 'miraculeuse', 'john', 'kramer', 'rend', 'mexique', 'procédure', 'médicale', 'risquée', 'expérimentale', 'découvrir']


In [43]:
len(set(corpus))

20602

### 3.4 Stemming 

In [45]:
stemmer = FrenchStemmer()

def stem_words(token_list):
    return [stemmer.stem(word) for word in token_list]

stem_words(corpus)

['group',
 'animal',
 'animatron',
 'interpret',
 'chanson',
 'enfant',
 'jour',
 'fait',
 'meurtri',
 'nuit',
 'adapt',
 'jeu',
 'vidéo',
 'fiv',
 'freddy',
 'crois',
 'action',
 'strateg',
 'espoir',
 'guérison',
 'miracul',
 'john',
 'kram',
 'rend',
 'mexiqu',
 'procédur',
 'médical',
 'risqu',
 'expérimental',
 'découvr',
 'tout',
 'oper',
 'arnaqu',
 'vis',
 'escroqu',
 'plus',
 'vulner',
 'armé',
 'nouvel',
 'object',
 'trist',
 'célebr',
 'tueur',
 'ser',
 'utilis',
 'pieg',
 'ingéni',
 'renvers',
 'situat',
 'contr',
 'escroc',
 'jak',
 'offici',
 'polic',
 'équip',
 'los',
 'angel',
 'voit',
 'partenair',
 'dévou',
 'mour',
 'abattu',
 'agresseur',
 'alor',
 'recherch',
 'ident',
 'tireur',
 'découvr',
 'vast',
 'réseau',
 'flic',
 'corrompus',
 'impos',
 'loi',
 'tout',
 'vill',
 'depuis',
 'déjà',
 'plusieur',
 'géner',
 'musiqu',
 'famill',
 'miguel',
 'vrai',
 'déchir',
 'jeun',
 'garçon',
 'dont',
 'rêv',
 'ultim',
 'deven',
 'musicien',
 'auss',
 'accompl',
 'idol',
 'e

XXXXXXXXX

In [ ]:
df['Syno psis'] = df['Synopsis'].str.lower() 

In [ ]:
tokenizer = nltk.RegexpTokenizer(r'\w+')
df_train['Synopsis'] = df_train['Synopsis'].apply(tokenizer.tokenize)

In [ ]:
#fonction pour supprimer les pronoms, articles, determinants etc 


stop_words = set(stopwords.words('french'))

def remove_stop_words(token_list):
    return [word for word in token_list if word not in stop_words]

In [ ]:
# Application de la fonction à la colonne Synopsis

df_train['Synopsis'] = df_train['Synopsis'].apply(remove_stop_words)

In [ ]:
#Création d'une liste de la colonne Synopsis
all_words = [word for token_list in df_train['Synopsis'] for word in token_list]

In [ ]:
filtered_frequencies = Counter(all_words)

# Count the frequencies of the remaining words
filtered_word_frequencies = Counter(filtered_frequencies)

most_common_words = filtered_word_frequencies.most_common(30) 

# Display the 10 most common words
print(most_common_words)

In [ ]:
#wordcloud

In [ ]:
#affiner la liste de stop words à partir des mots les + fréquents ? Genre "a", "plus", "où"....

In [ ]:
#stemming 

stemmer = FrenchStemmer()

def stem_words(token_list):
    return [stemmer.stem(word) for word in token_list]

df_train['Synopsis'] = df_train['Synopsis'].apply(stem_words)

In [ ]:
#Bag of words
model = Word2Vec(df_train['Synopsis'], vector_size=300, window=5, min_count=3, sg=1)

In [ ]:
model.train(df_train['Synopsis'], total_examples=len(df_train), epochs=10)

In [ ]:
similar_words = model.wv.most_similar('zomb')
similar_words

In [ ]:
# Vectorisation binaires des genres
print(len(df_train))
print(df_train['Genre'].apply(lambda x: isinstance(x, str)).sum())

In [ ]:
df_train['Genre'] = df_train['Genre'].apply(lambda x: eval(x) if isinstance(x, str) else x)
print(df_train['Genre'].apply(lambda x: isinstance(x, list)).sum())

In [ ]:
mlb = MultiLabelBinarizer()
genre_binarized = mlb.fit_transform(df_train['Genre'])

# Créer un DataFrame avec les résultats
genre_df = pd.DataFrame(genre_binarized, columns=mlb.classes_)

In [ ]:
genre_df.index = df_train.index

In [ ]:
df = pd.concat([df_train.drop('Genre', axis=1), genre_df], axis=1)
df

In [ ]:
### Création du modèle de recommandation
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['Synopsis'])

In [ ]:
# Calcul de la similarité cosinus
#cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [ ]:
df['Synopsis'] = df['Synopsis'].apply(lambda x: ' '.join(x))
count_matrix = CountVectorizer().fit_transform(df['Synopsis'])

In [ ]:
indices = pd.Series(df.index, index=df['Titre']).to_dict()

In [ ]:
cosine_sim = cosine_similarity(count_matrix)

In [ ]:
# Fonction pour obtenir des recommandations
def get_recommendations(title, cosine_sim, df_movies_md, num_of_recs=10):

    title = title.lower()
    if title not in df_movies_md['Titre'].str.lower().values:
        return f"Aucune recommandation trouvée pour: {title}"
    
    # Obtenir l'indice du film donné son titre
    idx = df_movies_md[df_movies_md['Titre'].str.lower() == title.lower()].index[0]

    # Obtenir les scores de similarité pour ce film avec tous les films
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Multiplier par la note du film pour chaque score
    sim_scores = [(i, score * (df_movies_md.iloc[i]['Note'] / 10)) for i, score in sim_scores]

    # Trier les films en fonction des scores calculés
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Obtenir les scores des n films les plus similaires
    sim_scores = sim_scores[1:num_of_recs+1]

    # Obtenir les indices de ces films
    movie_indices = [i[0] for i in sim_scores]

    # Retourner les films correspondants
    return df_movies_md.iloc[movie_indices]

In [ ]:
# Obtenir des recommandations pour un film donné
recommendations = get_recommendations("Coco", cosine_sim, indices)

# Afficher les recommandations
print(recommendations)
df[df['Titre'].str.contains("irréversible", case=False, na=False)]